<a href="https://colab.research.google.com/github/crisjosil/Sentinel_Earth_Engine/blob/master/My_S1_EE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Connecting GEE with Google Services

Install and authenticate to Earth Engine

In [0]:
!pip install earthengine-api #earth-engine Python API

In [0]:
!earthengine authenticate 

Authenticate to Google Drive and Colab

In [0]:
from google.colab import drive
drive.mount('/content/drive')

In [0]:
from google.colab import auth
auth.authenticate_user()

# Testing the software setup

In [0]:
# Earth Engine Python API
import ee
ee.Initialize()

Install and import the GIS tools to handle and visualice shapefiles and GeoJson data

In [0]:
!pip install geopandas
import geopandas
!pip install shapely
import shapely
!pip install geetools
from geetools import collection,tools, ui

In [7]:
import folium


# Define the URL format used for Earth Engine generated map tiles.
EE_TILES = 'https://earthengine.googleapis.com/map/{mapid}/{{z}}/{{x}}/{{y}}?token={token}'

print('Folium version: ' + folium.__version__)

Folium version: 0.8.3


# Mapdisplay: Display GEE objects using folium.

In [0]:
#@title Mapdisplay: Display GEE objects using folium.
def Mapdisplay(center, dicc, Tiles="OpensTreetMap",zoom_start=10):
    '''
    :param center: Center of the map (Latitude and Longitude).
    :param dicc: Earth Engine Geometries or Tiles dictionary
    :param Tiles: Mapbox Bright,Mapbox Control Room,Stamen Terrain,Stamen Toner,stamenwatercolor,cartodbpositron.
    :zoom_start: Initial zoom level for the map.
    :return: A folium.Map object.
    '''
    mapViz = folium.Map(location=center,tiles=Tiles, zoom_start=zoom_start)
    for k,v in dicc.items():
      if ee.image.Image in [type(x) for x in v.values()]:
        folium.TileLayer(
            tiles = EE_TILES.format(**v),
            attr  = 'Google Earth Engine',
            overlay =True,
            name  = k
          ).add_to(mapViz)
      else:
        folium.GeoJson(
        data = v,
        name = k
          ).add_to(mapViz)
    mapViz.add_child(folium.LayerControl())
    return mapViz

In [0]:
def folium_gee_map(image,vis_params=None,folium_kwargs={}):
    """
    Function to view Google Earth Engine tile layer as a Folium map.
    
    Parameters
    ----------
    image : Google Earth Engine Image.
    vis_params : Dict with visualization parameters.
    folium_kwargs : Keyword args for Folium Map.
    """
    
    # Get the MapID and Token after applying parameters
    image_info = image.getMapId(vis_params)
    mapid = image_info['mapid']
    token = image_info['token']
    folium_kwargs['attr'] = ('Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a> ')
    folium_kwargs['tiles'] = "https://earthengine.googleapis.com/map/%s/{z}/{x}/{y}?token=%s"%(mapid,token)
    
    return folium.Map(**folium_kwargs)

def folium_gee_layer(folium_map,image,vis_params=None,folium_kwargs={}):
    """
    Function to add Google Earth Engine tile layer as a Folium layer.
    
    Parameters
    ----------
    folium_map : Folium map to add tile to.
    image : Google Earth Engine Image.
    vis_params : Dict with visualization parameters.
    folium_kwargs : Keyword args for Folium Map.
    """
    
    # Get the MapID and Token after applying parameters
    image_info = image.getMapId(vis_params)
    mapid = image_info['mapid']
    token = image_info['token']
    folium_kwargs['attr'] = ('Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a> ')
    folium_kwargs['tiles'] = "https://earthengine.googleapis.com/map/%s/{z}/{x}/{y}?token=%s"%(mapid,token)
    
    layer = folium.TileLayer(**folium_kwargs)
    layer.add_to(folium_map)

# Filter the Sentinel 1 Data


In [0]:
#Parameters: DATES, DESCENDING OR DESCENDING, POLARISATION 
str2='2019-08-04'
str1='2019-07-25'
#Direction='ASCENDING'
Direction='DESCENDING'
date2 = ee.Date(str2)
date1 = ee.Date(str1)
ds = date2.difference(date1, 'day')

# Load an image collection, filtered so it's not too much data.
Sentinel1 = ee.ImageCollection('COPERNICUS/S1_GRD').filterDate(date1, date2).filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VV')).filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VH')).filter(ee.Filter.eq('instrumentMode', 'IW')).filter(ee.Filter.eq('orbitProperties_pass', Direction)).filter(ee.Filter.eq('relativeOrbitNumber_start', 142)); 


In [0]:
#Sentinel1 = ee.ImageCollection('COPERNICUS/S1_GRD').filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VV')).filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VH')).filter(ee.Filter.eq('instrumentMode', 'IW')).filter(ee.Filter.eq('orbitProperties_pass', Direction)).filter(ee.Filter.eq('relativeOrbitNumber_start', 142)); 
#July=ee.Filter.date('2019-07-01','2019-07-31')

# Compute the median in each band, each pixel.
# Band names are VH_median, VV_median
July=Sentinel1.reduce(ee.Reducer.median())
# Compute the ratio (difference in dB)
Ratio=July.select('VH_median').subtract(July.select('VV_median'))
July_RGB=July.addBands(Ratio)
#Image_july=ee.image(Sentinel1.filter(July).mean())

In [0]:
# The output is an Image.  Add it to the map.
vis_param = {'min': -25, 'max':0, 'bands': ['VV_median', 'VH_median', 'VH_median_1'],'gamma': 1.6}
#vis_param = {'bands': ['VH_median', 'VV_median', 'VH_median_1'], 'gamma': 1.6}



In [0]:
median_tk = July_RGB.getMapId(vis_param)

In [80]:
center = [-8.520166448964439,-78.79737094642212]
Mapdisplay(center,{'Sentinel 1':median_tk},zoom_start=12)

# Read a shapefile from the google drive and overlay it on a openStreet maps layer

In [0]:
geo_df = geopandas.read_file('/content/drive/My Drive/Shapefiles/Danper_Campositan_parcels_boundaries_lotes_EPSG 4326 - WGS 84.shp')
geo_df.plot()

In [82]:
m = folium.Map([-8.520166448964439,-78.79737094642212], zoom_start=12)
folium.GeoJson(geo_df).add_to(m)
m

# New Section

In [0]:
# The output is an Image.  Add it to the map.
#vis_param = {min: -10, max:0, 'bands': ['VH_median']}
rgb_vis   = {min: -25, max:5, bands: ['VV_median', 'VH_median', 'VH_median_1']}
median_tk = July_RGB.getMapId(rgb_vis)

center = [-8.520166448964439,-78.79737094642212]
Mapdisplay(center,{'S1':median_tk},zoom_start=9)

In [0]:
# The output is an Image.  Add it to the map.
vis_param = {min: [-25,-20,-25], max:[0,10,0], 'bands': ['VV_median', 'VH_median', 'Ratio']}
median_tk = July_RGB.getMapId(vis_param)

center = [-8.520166448964439,-78.79737094642212]
Mapdisplay(center,{'S1':median_tk},zoom_start=9)

In [0]:
print(median_tk)

In [0]:
bands=July.bandNames()
print('Band names: ', bands)

In [0]:
from shapely.geometry import MultiPolygon, Polygon

footprint = None
for i in geo_df['geometry']:
    footprint = i

In [0]:
# Add GEE Terrain Layer
m = folium.Map([-8.520166448964439,-78.79737094642212], zoom_start=12)
rgb_vis =    {min: -40, max:0}
#vis_params = {'min':0.0, 'max':3000, 'palette':'00FFFF,0000FF'}
folium_gee_map(July,vis_params=rgb_vis,folium_kwargs={})
#folium_gee_layer(m,July,vis_params=rgb_vis,folium_kwargs={'overlay':True,'name':'RGB'})



In [0]:
July=Sentinel1.reduce(ee.Reducer.median())
#Ratio=July.select('VH_median').subtract(July.select('VV_median'))
#July_RGB=July.addBands(Ratio)
vis_param = {min: [-25,-20,-25], max:[0,10,0], 'bands': ['VV_median', 'VH_median', 'angle_median']}
m = folium.Map([-8.520166448964439,-78.79737094642212], zoom_start=12)
folium_gee_layer(m,July,vis_params=vis_param,folium_kwargs={'overlay':True,'name':'RGB'})

In [0]:
!pip install geetools

In [0]:
from geetools import ui